In [1]:
import sys
import os
import datetime
import time
import copy
from dataclasses import dataclass, field, replace
from collections import deque
import random
from tqdm import tqdm_notebook as tqdm

In [10]:
###################
###   Classes   ###
###################
@dataclass
class Locomotive():
    '''
    A locomotive has certain attributes and actions.
    
    Locomomtives have a name (str), class (str), weight (int),
    power (int), and max speed (int).
    
    There is currently no validation on these properties. When creating
    a consist, you should use consistent units.
    '''
    name:   str = field(default='Loco')
    number: int = field(default=1)
    road:   str = field(default='None')
    lclass: str = field(default='Steam')
    length: float = field(default=1.0, metadata={'units':'m'})
    mass:   float = field(default=1.0, metadata={'units':'kg'})
    power:  float = field(default=1.0, metadata={'units':'kW'})
    maxspeed: float = field(default=999.0, metadata={'units':'km/h'})
        
    
    def info(self):
        return 'Locomotive({}: {} #{}, {})'.format(self.name, self.road, self.number, self.lclass)
        
        
    def __str__(self):
        out = ''
        for k,v in self.__dict__.items():
            out += '\n{}: {}'.format(k.title(), v)
            
        return out
    

@dataclass
class RollingStock():
    '''
    A car holding goods, which can be pulled by a locomotive.
    
    Rolling Stock have a type (str), Railroad owner (str), number (int), length (int),
    dry mass (int), total capacity (int), current load mass (int), and a max speed (int).
    
    load_mass and maxspeed are optional, and defaulted to 0 (empty), and 1000 (no speed limit).
    
    There is currently no validation on these properties. When creating
    a consist, you should use consistent units.
    '''
    car_type: str = field(default='None')
    number: int = field(default=1)
    road: str = field(default=None)
    color: str = field(default='black')
    length: float = field(default=1.0, metadata={'units':'m'})
    dry_mass: float = field(default=1.0, metadata={'units':'kg'})
    capacity: float = field(default=1.0, metadata={'units':'kg'})
    load_mass: float = field(default=0.0, metadata={'units':'kg'})
    maxspeed: float = field(default=999.0, metadata={'units':'km/h'})
    
    
    def info(self):
        return '{}: {} #{}, {}'.format(self.car_type, self.road, self.number, self.color)
        
    
    def __str__(self):
        '''
        Returns a string which can be easily printed to show all current properties of the rolling stock.
        '''
        out = ''
        for k,v in self.__dict__.items():
            out += '\n{}: {}'.format(k.title(), v)
            
        return out
    
    
    @property
    def mass(self):
        '''
        Property of a rolling stock car which represents the total weight at the current time.
        '''
        return self.dry_mass + self.load_mass
        
    
    @property
    def isfull(self):
        '''
        Property of a rolling stock car which represents how much cargo it currently holds.
        Returns percent of load relative to total capacity (0 to 1).
        '''
        return (self.mass - self.dry_mass)/self.capacity
    
    
    def load(self, cargo):
        '''
        Load an amount of mass in the rolling stock.
        
        Returns 0 if all mass is loaded, remaining mass if excess.
        '''
        # If the mass to load is less than the remaining capacity
        if cargo <= (1-self.isfull)*self.capacity:
            # Load all the mass
            self.load_mass += cargo
            return 0
        else:
            # Store the available mass for return operation
            e = (1-self.isfull)*self.capacity
            # Fill up the cargo
            self.load_mass = self.capacity
            # Return the difference
            return cargo - e
        
        
    def unload(self, p=1) -> float:
        '''
        Unloads a percentage of mass from the rolling stock.
        Default to fully unload.
        
        Returns the mass removed.
        '''
        if p > self.isfull:
            p = self.isfull
        
        mass = p*(self.load_mass-self.dry_mass)
        self.load_mass -= mass
        return mass
        
    
@dataclass
class Consist():
    '''
    A Consist is a set of train classes. This can or cannot include a locomotive.
    Order matters in a consist.
    '''    
    number: int = field(default=1)
    stock: deque = field(default=deque())
    
    
    def __post_init__(self, *argv):
        # The stock of a consist contains all the cars and locomotives, and their order.
        
        # If only 1 additional argument is given,
        # assume it's a deque to be used as the new stock
        if len(argv) == 1:
            self.stock = deque(argv[0])
        else:
            self.stock = deque(argv)


    def __str__(self):
        out = '< Consist #{}\n'.format(self.number)
        if len(self.stock) == 0:
            out = 'Consist {} is empty! Add some cars or maybe delete it?'.format(self.number)
        else:
            for car in self.stock:
                out += '{}\n'.format(car.info())
                
        out += '>'
                
        return out

    
    def info(self):
        '''
        This method provides load information for each car in the consist.
        '''
        out = ''
        if len(self.stock) > 0:
            for car in self.stock:
                if isinstance(car, Locomotive):
                    continue

                if car.isfull == 0:
                    out += '{} {} is empty.\n'.format(car.car_type, car.number)
                else:
                    out += '{} {} is {:.1%} full.\n'.format(car.car_type, car.number, car.isfull)
        else:
            out = 'Consist {} is empty! Add some cars or maybe delete it?'.format(self.number)
        
        return out
    
    
    @property
    def length(self):
        '''
        Consist length property.
        '''
        length = 0
        for car in self.stock:
            length += car.length
            
        return length
            
    
    @property
    def mass(self):
        '''
        Consist total mass property.
        '''
        mass = 0
        for car in self.stock:
            mass += car.mass
        
        return mass
            
    
    def attach(self, car):
        '''
        This method attaches a car at the end of the consist stock.
        '''
        self.stock.append(car)
        
    
    def separate(self, car):
        '''
        This method splits the consist in two. The car provided
        will be the lead car in the new consist (aka separate the consist
        _before_ the provided car).
        '''
        idx = self.stock.index(car)
        self.stock.rotate(-idx)
        
        new_stock = deque()
        for i in range(len(self.stock)-idx):
            new_stock.append(self.stock.popleft())
            
        return Consist(self.number+1, new_stock)



#######################
###   Sub-Classes   ###
#######################
@dataclass
class BigBoy(Locomotive):
    '''
    The Union Pacific Big Boy is a type of simple articulated 4-8-8-4 steam locomotive
    manufactured by the American Locomotive Company between 1941 and 1944 and operated
    by the Union Pacific Railroad in revenue service until 1959.
    
    The 25 Big Boy locomotives were built to haul freight over the Wasatch mountains
    between Ogden, Utah, and Green River, Wyoming. In the late 1940s, they were reassigned
    to Cheyenne, Wyoming, where they hauled freight over Sherman Hill to Laramie, Wyoming.
    They were the only locomotives to use a 4-8-8-4 wheel arrangement: four-wheel leading
    truck for stability entering curves, two sets of eight driving wheels and a four-wheel
    trailing truck to support the large firebox.
    
    Eight Big Boys survive, most on static display at museums across the country.
    One of them, No. 4014, was re-acquired by Union Pacific and restored to operating
    condition in 2019, regaining the title as the largest and most powerful operating
    steam locomotive in the world.
    '''
    name: str = field(default='UP_BigBoy')
    road: str = field(default='UP')
    lclass: str = field(default='Steam')
    length: float = field(default=40.47, metadata={'units':'m'})
    mass: float = field(default=567000, metadata={'units':'kg'})
    power: float = field(default=5200, metadata={'units':'kW'})
    maxspeed: float = field(default=130, metadata={'units':'km/h'})


@dataclass
class Boxcar(RollingStock):
    '''
    A basic boxcar.
    '''
    car_type: str = field(default='Boxcar')
    length: float = field(default=15.0, metadata={'units':'m'})
    dry_mass: float = field(default=22600, metadata={'units':'kg'})
    capacity: float = field(default=45.4, metadata={'units':'kg'})
        

@dataclass
class Gondola(RollingStock):
    '''
    A basic gondola car.
    '''
    car_type: str = field(default='Gondola')
    length: float = field(default=15.0, metadata={'units':'m'})
    dry_mass: float = field(default=22600, metadata={'units':'kg'})
    capacity: float = field(default=45.4, metadata={'units':'kg'})
        
        
@dataclass
class Hopper(RollingStock):
    '''
    A basic hopper car.
    '''
    car_type: str = field(default='Hopper')
    length: float = field(default=15.0, metadata={'units':'m'})
    dry_mass: float = field(default=22600, metadata={'units':'kg'})
    capacity: float = field(default=45.4, metadata={'units':'kg'})
        
        
@dataclass
class Flatcar(RollingStock):
    '''
    A basic flatcar.
    '''
    car_type: str = field(default='Flatcar')
    length: float = field(default=15.0, metadata={'units':'m'})
    dry_mass: float = field(default=22600, metadata={'units':'kg'})
    capacity: float = field(default=45.4, metadata={'units':'kg'})



############################
###   Global Variables   ###
############################
COLORS = ['red', 'blue', 'yellow', 'green', 'brown', 'white', 'grey', 'black', 'orange']
ROADS = ['UP', 'NO', 'SOO', 'CSX', 'CC', 'TS', 'BNSF', 'ICG', 'ED&T']
TYPS = ['One']
TYPS = [Boxcar, Gondola, Hopper, Flatcar]
LOAD_TIME = 0.05       # How long (in real seconds) it takes to load 1 unit of cargo in 1 car.


#####################
###   Functions   ###
#####################
def load_consist(cargo, con, typ=None):
    '''
    This function will take an input cargo stock, and consist, and load
    each car in the consist to full, until entire consist is full, or
    no cargo remains.
    
    Optionally, it can be set to only load on cars in the consist which have
    a matching car_type attribute.
    
    Returns remaining cargo mass, and loaded consist.
    '''
    if typ is None:
        for car in tqdm(con.stock):
            if isinstance(car, Locomotive):
                continue
                
            if cargo == 0:
                return cargo, con
            else:
                time.sleep(LOAD_TIME)
                cargo = car.load(cargo)
        
        return cargo, con
    else:
        for car in tqdm(con.stock):
            if isinstance(car, Locomotive):
                continue
            
            if cargo == 0:
                return cargo, con
            elif car.car_type == typ.title():
                time.sleep(LOAD_TIME)
                cargo = car.load(cargo)
        
        return cargo, con
    

def random_consist(loco,
                   num,
                   carrange: tuple = (1,20),
                   numrange: tuple = (100,99999),
                   colors = COLORS,
                   roads = ROADS,
                   car_types = TYPS
                  ):
    '''
    Generates a random consist with the specified variables.
    
    Returns the consist, and prints results.
    '''
    train = Consist(num)
    train.attach(loco)
    for i in range(carrange[0], carrange[1]):
        num = random.randint(numrange[0], numrange[1])
        color = random.choice(colors)
        road = random.choice(roads)
        car = random.choice(car_types)
        train.attach(car(road=road, number=num, color=color))

    print('')
    print(train)
    print('')
    print('Consist Length: {:.2f}[m]'.format(train.length))
    print('Consist Mass: {:.3f}[kg]'.format(train.mass))
    
    return train


def sort_consist(con: Consist, typs, roads) -> Consist:
    '''
    This function sorts a consist so all cars of equal types/roads are
    grouped together.
    
    Note this isn't very realistic, so it's mainly for consist generation,
    and thus why it's not included as a class method.
    
    Inputs are consist to be sorted, and list of types to sort. Unsorted
    types are moved to the end, and types are sorted in the order provided.
    
    Returns the sorted consist.
    '''
    # Initialize the output
    con_sorted = Consist(con.number)
    # Move the locos to the head of the output
    for car in con.stock:
        if isinstance(car, Locomotive):
            con_sorted.attach(car)

           
    for road in ROADS:
        for t in typs:
            for car in con.stock:
                if isinstance(car, t) and car.road == road:
                    con_sorted.attach(car)
                
    
    return con_sorted

In [11]:
rock = 220    # [kg]
loco = BigBoy(number=4004)
train = random_consist(loco, random.randint(0,9999))


< Consist #1820
Locomotive(UP_BigBoy: UP #4004, Steam)
Hopper: BNSF #71181, blue
Flatcar: BNSF #51564, yellow
Gondola: SOO #10827, green
Boxcar: ICG #80950, grey
Gondola: CC #16478, black
Hopper: UP #68670, red
Flatcar: SOO #69213, brown
Boxcar: NO #3716, black
Gondola: CC #60413, brown
Boxcar: SOO #68269, grey
Boxcar: BNSF #98747, white
Boxcar: ICG #44812, red
Hopper: BNSF #22085, blue
Gondola: CSX #5638, red
Boxcar: ED&T #46743, red
Boxcar: TS #85143, orange
Gondola: NO #45113, black
Hopper: CSX #44417, yellow
Hopper: TS #55743, green
>

Consist Length: 325.47[m]
Consist Mass: 996400.000[kg]


In [13]:
train = sort_consist(train, TYPS, ROADS)
print(train)
print('')
print('Consist Length: {:.2f}[m]'.format(train.length))
print('Consist Mass: {:.3f}[kg]'.format(train.mass))

< Consist #1820
Locomotive(UP_BigBoy: UP #4004, Steam)
Hopper: UP #68670, red
Boxcar: NO #3716, black
Gondola: NO #45113, black
Boxcar: SOO #68269, grey
Gondola: SOO #10827, green
Flatcar: SOO #69213, brown
Gondola: CSX #5638, red
Hopper: CSX #44417, yellow
Gondola: CC #16478, black
Gondola: CC #60413, brown
Boxcar: TS #85143, orange
Hopper: TS #55743, green
Boxcar: BNSF #98747, white
Hopper: BNSF #71181, blue
Hopper: BNSF #22085, blue
Flatcar: BNSF #51564, yellow
Boxcar: ICG #80950, grey
Boxcar: ICG #44812, red
Boxcar: ED&T #46743, red
>

Consist Length: 325.47[m]
Consist Mass: 996400.000[kg]


In [16]:
print('Loading rock in gondolas only.')
rock, train = load_consist(rock, train, typ='gondola')
print('Rock remaining after consist load: {:.3f}[kg]'.format(rock))
print('')

print(train.info())
        
print('')
print('After loading, consist weighs: {:.3f}[kg]'.format(train.mass))

Loading rock in gondolas only.



Rock remaining after consist load: 0.000[kg]

Hopper 68670 is empty.
Boxcar 3716 is empty.
Gondola 45113 is 100.0% full.
Boxcar 68269 is empty.
Gondola 10827 is 100.0% full.
Flatcar 69213 is empty.
Gondola 5638 is 100.0% full.
Hopper 44417 is empty.
Gondola 16478 is 100.0% full.
Gondola 60413 is 84.6% full.
Boxcar 85143 is empty.
Hopper 55743 is empty.
Boxcar 98747 is empty.
Hopper 71181 is empty.
Hopper 22085 is empty.
Flatcar 51564 is empty.
Boxcar 80950 is empty.
Boxcar 44812 is empty.
Boxcar 46743 is empty.


After loading, consist weighs: 996620.000[kg]
